In [3]:
from pyspark.sql import functions as F, SparkSession

In [4]:
# Create a Spark Session
spark = (
    SparkSession.builder.appName("consumer model")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.execturo.memory", "2g")
    .getOrCreate()
)

24/09/08 10:26:49 WARN Utils: Your hostname, qinsitaodeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 100.92.15.134 instead (on interface en0)
24/09/08 10:26:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/08 10:26:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Load data
consumer_info = spark.read.parquet('../data/curated/consumer_info.parquet')
transaction_records = spark.read.parquet('../data/curated/transaction_records.parquet')
fraudulent_consumer_rate = spark.read.parquet('../data/curated/consumer_fraud_rate.parquet')

In [6]:
fraudulent_consumer_with_info = consumer_info.join(fraudulent_consumer_rate, on="consumer_id", how="inner")


In [7]:
# Average fraud probability in each postcode or state
fraudulent_consumer_group_by_postcode = fraudulent_consumer_with_info.groupBy(["postcode"]).agg(F.avg("fraud_probability").alias("average_fraud_prob_of_postcode"))

fraudulent_consumer_group_by_state = fraudulent_consumer_with_info.groupBy(["state"]).agg(F.avg("fraud_probability").alias("average_fraud_prob_of_state"))

In [8]:
# Get average fraud prob for each consumer
average_fraudulent_consumer_rate = fraudulent_consumer_rate.groupBy("consumer_id").agg(F.avg("fraud_probability").alias("average_fraud_probability"))

In [9]:
fraudulent_consumer_rate.count()

34864

In [10]:
# number of unique available fraudulent consumer
average_fraudulent_consumer_rate.where(average_fraudulent_consumer_rate["average_fraud_probability"]>0).count()

20128

In [11]:
# Add consumer info to transaction records
consumer_transaction_records = transaction_records.join(consumer_info, on="consumer_id", how="inner")

In [12]:
# order has really high variance and value
consumer_transaction_records.select("dollar_value").summary()

24/09/08 10:27:07 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


summary,dollar_value
count,14195505
mean,166.22895390891753
stddev,517.8505602612809
min,9.756658099412162E-8
25%,26.128899475514228
50%,62.22903376686649
75%,150.41282294992547
max,105193.88578925544


In [13]:
# analysis order value, consider the variance of order value and purchase frequency
consumer_transaction_value_analysis =  consumer_transaction_records.groupBy("consumer_id", "state", "postcode") \
                                        .agg(
                                            F.avg("dollar_value").alias("average_dollar_value"),
                                            F.min("dollar_value").alias("min_dollar_value"),
                                            F.max("dollar_value").alias("max_dollar_value"),
                                            F.count("dollar_value").alias("transaction_count"),
                                            F.stddev("dollar_value").alias("stddev_dollar_value")
                                        )

In [14]:
# Prepare data frame for modelling
fraudulent_consumer_summary = average_fraudulent_consumer_rate \
    .join(consumer_transaction_value_analysis, on="consumer_id", how="left") \
    .join(fraudulent_consumer_group_by_postcode, on="postcode", how="inner") \
    .join(fraudulent_consumer_group_by_state, on="state", how="inner")

In [42]:
fraudulent_consumer_summary.show(3)


+-----+--------+-----------+-------------------------+--------------------+--------------------+------------------+-----------------+-------------------+------------------------------+---------------------------+
|state|postcode|consumer_id|average_fraud_probability|average_dollar_value|    min_dollar_value|  max_dollar_value|transaction_count|stddev_dollar_value|average_fraud_prob_of_postcode|average_fraud_prob_of_state|
+-----+--------+-----------+-------------------------+--------------------+--------------------+------------------+-----------------+-------------------+------------------------------+---------------------------+
|  VIC|    3171|    1463154|       11.878266056831936|  187.92238278341324|0.002626295938134...| 9383.286300772766|              578|   601.940558510518|            11.643992863529116|         15.162124631050172|
|   NT|     810|     240762|       10.591450552633916|  187.84754214087144|0.043988124325446215| 4405.580619592672|              591| 434.1325181138

# Idea
1. Time Frequency feature: https://ieeexplore.ieee.org/document/9399421/

# Modelling

In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

In [43]:
# List of features to be used in the model
features = [
    "average_dollar_value", "min_dollar_value", "max_dollar_value", 
    "transaction_count", "stddev_dollar_value", 
    "average_fraud_prob_of_postcode", "average_fraud_prob_of_state"
]

# VectorAssembler to combine the features into a single vector
assembler = VectorAssembler(inputCols=features, outputCol="features")

# Prepare the data
data = assembler.transform(fraudulent_consumer_summary)

train_data, test_data = data.randomSplit([0.8, 0.2])


In [44]:
train_data.select("features").show(1, truncate=False)

+----------------------------------------------------------------------------------------------------------------------+
|features                                                                                                              |
+----------------------------------------------------------------------------------------------------------------------+
|[169.66993713901135,0.4200595653096262,8936.232778956677,588.0,487.0941673231385,13.054126033171633,14.40842251077964]|
+----------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [45]:
# Define model regressor
dt = DecisionTreeRegressor(labelCol="average_fraud_probability", featuresCol="features")

rf = RandomForestRegressor(labelCol="average_fraud_probability", featuresCol="features")


In [46]:
# Parameter grid
dt_param_grid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [3, 5, 7]) \
    .addGrid(dt.maxBins, [32, 64]) \
    .build()


rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 7]) \
    .build()


In [63]:
# Evaluator for regression models
rmse_evaluator = RegressionEvaluator(
    labelCol="average_fraud_probability", 
    predictionCol="prediction", 
    metricName="rmse"  
)

r2_evaluator = RegressionEvaluator(
    labelCol="average_fraud_probability",
    predictionCol="prediction",
    metricName="r2" 
)

# Cross-validation 
dt_cv = CrossValidator(
    estimator=dt,
    estimatorParamMaps=dt_param_grid,
    evaluator=r2_evaluator,
    numFolds=3
)

rf_cv = CrossValidator(
    estimator=rf,
    estimatorParamMaps=rf_param_grid,
    evaluator=r2_evaluator,
    numFolds=3
)


In [64]:
# Pipeline 
dt_pipeline = Pipeline(stages=[dt_cv])

rf_pipeline = Pipeline(stages=[rf_cv])


In [ ]:
# Fit model

# 6 mins
#dt_model = dt_pipeline.fit(train_data)

rf_model = rf_pipeline.fit(train_data)


KeyboardInterrupt: 

# Evaluation

In [ ]:
# Make predictions on the test data
#dt_predictions = dt_model.transform(test_data)
rf_predictions = rf_model.transform(test_data)

# dt_rmse = rmse_evaluator.evaluate(dt_predictions)
# dt_r2 = r2_evaluator.evaluate(dt_predictions)
# print(f"Decision Tree RMSE: {dt_rmse}")
# print(f"Decision Tree R2: {dt_r2}")

rf_rmse = rmse_evaluator.evaluate(rf_predictions)
rf_r2 = r2_evaluator.evaluate(rf_predictions)
print(f"Random Forest RMSE: {rf_rmse}")
print(f"Random Forest R2: {rf_r2}")


Decision Tree RMSE: 6.9413395371206965
Decision Tree R2: 0.4556215686221997


In [ ]:
# Best model hyperparameters
# best_dt_model = dt_model.stages[-1].bestModel
# print(f"Best Decision Tree maxDepth: {best_dt_model._java_obj.getMaxDepth()}")
# print(f"Best Decision Tree maxBins: {best_dt_model._java_obj.getMaxBins()}")


# best_rf_model = rf_model.stages[-1].bestModel
# print(f"Best Random Forest numTrees: {best_rf_model.getNumTrees}")
# print(f"Best Random Forest maxDepth: {best_rf_model.getMaxDepth()}")


Best Decision Tree maxDepth: 5
Best Decision Tree maxBins: 64
